In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gensim transformers torch scipy pandas nltk

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
class WordSimilarityEvaluator:
    def __init__(self):
        print("Đang tải mô hình Word2Vec... (có thể mất vài phút)")
        self.word2vec_model = api.load('word2vec-google-news-300')

        print("Đang tải mô hình BERT...")
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

        print("Đang tải mô hình Transformer...")
        # Use a general transformer model (e.g., BERT) instead of CLIP
        self.transformer_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.transformer_model = BertModel.from_pretrained('bert-base-uncased')

    def load_dataset_from_file(self, file_path):
        """Tải dữ liệu từ file CSV hoặc Excel"""
        if file_path.endswith(".csv"):
            data = pd.read_csv(file_path)
        elif file_path.endswith((".xls", ".xlsx")):
            data = pd.read_excel(file_path)
        else:
            raise ValueError("File phải có định dạng .csv hoặc .xlsx!")

        # Kiểm tra các cột cần thiết
        if not {'word1', 'word2', 'human_score'}.issubset(data.columns):
            raise ValueError("File phải chứa các cột: 'word1', 'word2', 'human_score'")

        return data

    def get_word2vec_similarity(self, word1, word2):
        try:
            return self.word2vec_model.similarity(word1.lower(), word2.lower())
        except KeyError:
            return None

    def get_bert_similarity(self, word1, word2):
        inputs1 = self.bert_tokenizer(word1, return_tensors="pt", truncation=True, max_length=512)
        inputs2 = self.bert_tokenizer(word2, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs1 = self.bert_model(**inputs1)
            outputs2 = self.bert_model(**inputs2)
        embedding1 = outputs1.last_hidden_state.mean(dim=1)
        embedding2 = outputs2.last_hidden_state.mean(dim=1)
        return self._cosine_similarity(embedding1, embedding2)

    def get_transformer_similarity(self, word1, word2):
        # Tokenize and get embeddings from the transformer model (BERT in this case)
        inputs1 = self.transformer_tokenizer(word1, return_tensors="pt", truncation=True, max_length=512)
        inputs2 = self.transformer_tokenizer(word2, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs1 = self.transformer_model(**inputs1)
            outputs2 = self.transformer_model(**inputs2)
        embedding1 = outputs1.last_hidden_state.mean(dim=1)
        embedding2 = outputs2.last_hidden_state.mean(dim=1)
        return self._cosine_similarity(embedding1, embedding2)

    def _cosine_similarity(self, vec1, vec2):
        return torch.nn.functional.cosine_similarity(vec1, vec2, dim=1).item()

    def evaluate_from_file(self, file_path):
        """Đánh giá từ file dữ liệu"""
        data = self.load_dataset_from_file(file_path)

        # Thêm cột kết quả cho từng mô hình
        data['Word2Vec_Similarity'] = data.apply(lambda row: self.get_word2vec_similarity(row['word1'], row['word2']), axis=1)
        data['BERT_Similarity'] = data.apply(lambda row: self.get_bert_similarity(row['word1'], row['word2']), axis=1)
        data['Transformer_Similarity'] = data.apply(lambda row: self.get_transformer_similarity(row['word1'], row['word2']), axis=1)

        # Loại bỏ các hàng có giá trị None
        data = data.dropna(subset=['Word2Vec_Similarity', 'BERT_Similarity', 'Transformer_Similarity'])

        # Tính tương quan Spearman cho từng mô hình
        w2v_corr = stats.spearmanr(data['Word2Vec_Similarity'], data['human_score'])
        bert_corr = stats.spearmanr(data['BERT_Similarity'], data['human_score'])
        transformer_corr = stats.spearmanr(data['Transformer_Similarity'], data['human_score'])

        # In hệ số tương quan vào bảng
        data['Word2Vec_Correlation'] = w2v_corr.correlation
        data['BERT_Correlation'] = bert_corr.correlation
        data['Transformer_Correlation'] = transformer_corr.correlation

        return data

In [ ]:
# import gensim.downloader as api
# import os

# def download_word2vec_model(model_name='word2vec-google-news-300', save_path='word2vec_model'):
#     """
#     Tải và lưu model Word2Vec

#     Parameters:
#     - model_name: Tên model muốn tải (mặc định là 'word2vec-google-news-300')
#     - save_path: Đường dẫn để lưu model
#     """
#     # Tạo thư mục nếu chưa tồn tại
#     os.makedirs(save_path, exist_ok=True)

#     # Tải model
#     print(f"Đang tải model {model_name}... (có thể mất vài phút)")
#     model = api.load(model_name)

#     # Đường dẫn file lưu
#     file_path = os.path.join(save_path, f'{model_name}.bin')

#     # Lưu model
#     model.save(file_path)
#     print(f"Đã lưu model tại: {file_path}")

# def load_saved_word2vec_model(file_path):
#     """
#     Tải lại model Word2Vec đã lưu

#     Parameters:
#     - file_path: Đường dẫn đến file model đã lưu

#     Returns:
#     - Model Word2Vec đã được tải
#     """
#     from gensim.models import KeyedVectors

#     print(f"Đang tải model từ: {file_path}")
#     model = KeyedVectors.load(file_path)
#     return model

# # Ví dụ sử dụng
# if __name__ == "__main__":
#     # Tải và lưu model
#     download_word2vec_model()

#     # Nạp lại model đã lưu
#     loaded_model = load_saved_word2vec_model('word2vec_model/word2vec-google-news-300.bin')

#     # Kiểm tra model bằng cách tìm các từ tương đồng
#     try:
#         similar_words = loaded_model.most_similar('computer')
#         print("\nCác từ tương đồng với 'computer':")
#         for word, score in similar_words:
#             print(f"{word}: {score}")
#     except Exception as e:
#         print("Lỗi khi tìm từ tương đồng:", e)

Đang tải model word2vec-google-news-300... (có thể mất vài phút)
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Đã lưu model tại: word2vec_model/word2vec-google-news-300.bin
Đang tải model từ: word2vec_model/word2vec-google-news-300.bin

Các từ tương đồng với 'computer':
computers: 0.7979379892349243
laptop: 0.6640493273735046
laptop_computer: 0.6548868417739868
Computer: 0.647333562374115
com_puter: 0.6082080006599426
technician_Leonard_Luchko: 0.5662748217582703
mainframes_minicomputers: 0.5617720484733582
laptop_computers: 0.5585449934005737
PC: 0.5539618730545044
maker_Dell_DELL.O: 0.5519254207611084


In [ ]:
# Đường dẫn tới file CSV hiện tại
file_path = "/content/drive/MyDrive/DATN_LTN/rg65.csv"
output_file = "/content/drive/MyDrive/DATN_LTN/rg65_withHeader.csv"

# Đọc file CSV với dấu phân cách chính xác
df = pd.read_csv(file_path, sep=";", header=None)  # Thử với dấu phẩy (",")

# Kiểm tra nếu file vẫn không đúng, thử dấu phân cách khác (ví dụ tab "\t")
if df.shape[1] == 1:  # Nếu chỉ có 1 cột, file có thể dùng dấu phân cách khác
    df = pd.read_csv(file_path, sep="\t", header=None)

# Gán tiêu đề chính xác
df.columns = ["word1", "word2", "human_score"]

# Lưu file mới
df.to_csv(output_file, index=False)
print(f"File đã được sửa và lưu tại: {output_file}")


File đã được sửa và lưu tại: /content/drive/MyDrive/DATN_LTN/rg65_withHeader.csv


###Bộ dữ liệu RG65

In [ ]:
from google.colab import files
# Sử dụng lớp và phương thức đã định nghĩa
evaluator = WordSimilarityEvaluator()

# Tải dữ liệu từ file
file_path = "/content/drive/MyDrive/DATN_LTN/rg65_withHeader.csv"
data = evaluator.evaluate_from_file(file_path)

# In kết quả ra bảng
print(data)

# Lưu kết quả ra CSV
data.to_csv("similarity_results.csv", index=False)
files.download("similarity_results.csv")

Đang tải mô hình Word2Vec... (có thể mất vài phút)
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Đang tải mô hình BERT...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Đang tải mô hình Transformer...
         word1      word2  human_score  Word2Vec_Similarity  BERT_Similarity  \
0         cord      smile         0.02             0.018116         0.813448   
1      rooster     voyage         0.04             0.062758         0.771054   
2         noon     string         0.04             0.021655         0.665650   
3        fruit    furnace         0.05             0.073215         0.794024   
4    autograph      shore         0.06             0.034656         0.597448   
..         ...        ...          ...                  ...              ...   
60     cushion     pillow         3.84             0.251615         0.780219   
61    cemetery  graveyard         3.88             0.642481         0.770382   
62  automobile        car         3.92             0.583837         0.844061   
63      midday       noon         3.94             0.552741         0.914311   
64         gem      jewel         3.94             0.621081         0.904897   

    Tra

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

###Bộ dữ liệu WorSim

In [5]:
from google.colab import files
# Sử dụng lớp và phương thức đã định nghĩa
evaluator = WordSimilarityEvaluator()

# Tải dữ liệu từ file
file_path = "/content/drive/MyDrive/DATN_LTN/wordsim353crowd.csv"
data = evaluator.evaluate_from_file(file_path)

# In kết quả ra bảng
print(data)

# Lưu kết quả ra CSV
data.to_csv("similarity_results2.csv", index=False)
files.download("similarity_results2.csv")

Đang tải mô hình Word2Vec... (có thể mất vài phút)
Đang tải mô hình BERT...
Đang tải mô hình Transformer...
            word1       word2  human_score  Word2Vec_Similarity  \
0       admission      ticket       5.5360             0.436883   
1         alcohol   chemistry       4.1250             0.088762   
2        aluminum       metal       6.6250             0.637576   
3    announcement      effort       2.0625             0.231306   
4    announcement        news       7.1875             0.467333   
..            ...         ...          ...                  ...   
348        weapon      secret       2.5000             0.258439   
349       weather    forecast       5.4375             0.362721   
350     Wednesday        news       1.1250             0.177296   
351          wood      forest       7.9375             0.417516   
352          word  similarity       0.8125             0.096587   

     BERT_Similarity  Transformer_Similarity  Word2Vec_Correlation  \
0           0.842

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>